In [2]:
import igl
#import meshplot as mp
#import vedo as vd
import polyscope as ps
import numpy as np
import os 
import time 
from geometry.mesh import Mesh
from geometry.utils import *
from optimization.Planarity import Planarity
from optimization.HyperbolicLC import HyperbolicLC
from optimization.Optimizer import Optimizer
from optimization.LineCong import LineCong

#vd.settings.default_backend = 'k3d'

## Initial Line Congruence test

### Visualization setup

In [ ]:
dir_path = os.getcwd()

data_path = dir_path+"/approximation/data/"

mv, mf = igl.read_triangle_mesh(os.path.join(data_path,"centers.obj" ))
tv, tf = igl.read_triangle_mesh(os.path.join(data_path,"test_remeshed.obj" ))
# Get vertex normals for test mesh
tn = igl.per_vertex_normals(tv, tf)

signs = np.sign(np.sum(tn * ([0,0,1]), axis=1))

# Fix normal directions
tn = tn * signs[:, None]

# Compute circumcenters and axis vectors for each triangle
p1, p2, p3 = tv[tf[:, 0]], tv[tf[:, 1]], tv[tf[:, 2]]

ct, rt, nt = circle_3pts(p1, p2, p3)

# Create dual mesh
mesh = Mesh()

mesh.make_mesh(tv,tf)

# Get inner vertices
inner_vertices = mesh.inner_vertices()

# Dual topology including outer faces
dual = mesh.dual_top()

# Dual topology without outer faces
dual_faces = [dual[vertex] for vertex in inner_vertices ]


# dual_faces = dualmesh.faces()
# # Draw vertex normals
# # vd.show(vd.Points(tv, c='red'), vd.Arrows(tv, tv + tn * 0.1, c='blue'))

# # Draw circles
# #circles = [ vd.Line(vd.Circle(ct, rt, c='k').orientation(nt), closed=True, lw=1.2, c='black') for ct, rt, nt in zip(ct, rt, nt) ]

# Create test mesh and center mesh 
mesh =  vd.Mesh((tv, tf), alpha = 0.8)
center = vd.Mesh((mv, mf), alpha = 0.9, c=[0.4, 0.4, 0.81])

# Create hexagonal mesh                            
h_pts = np.empty((len(tf), 3), dtype=np.float64)

# Intersect circumcircle axis with center mesh
for i in range(len(tf)):
    # Get points on circumcircle axis
    p0  = ct[i] - 10*nt[i]
    p1  = ct[i] + 10*nt[i]
    
    # Get intersection points
    h_pts[i,:] = np.array(center.intersect_with_line(p0, p1)[0])

# Create hexagonal mesh
hexmesh = vd.Mesh((h_pts, dual_faces), alpha = 0.5, c=[0.2, 0.4, 0.21])

# Draw hexagonal mesh
linesHex = []

for face in dual_faces:
        hex_face = []
        for vi in face:
            hex_face.append(h_pts[vi])
        linesHex.append(vd.Line(hex_face, closed=True, lw=1.8, c='white'))
            
# Draw edges center mesh
center_edges = center.clone().wireframe().lw(1.5).flat().c('black').alpha(1)

# Draw edges Triangular mesh
edges = mesh.clone().lw(1.5).wireframe().flat().c('k').alpha(0.5)

# Draw axis lines
# Lines vd.Lines(ct- 5*nt, ct + 5*nt, c='red', lw=0.1)
clines = vd.Lines(ct[:5]- 5*nt[:5], ct[:5] + 5*nt[:5], c='tomato', lw=0.1)

vd.show(vd.Points(tv, c='red'), mesh, center, vd.Points(h_pts, c='red'), linesHex, edges, clines)
 

### Optimization Test

In [ ]:
# Define paths
dir_path = os.getcwd()
data_path = dir_path+"/approximation/data/" # data path
out_path = dir_path+"/outputs/" # output path

# Data of interest
k = 2

# Init data files
file_pts = open(os.path.join( out_path, "points_"+str(k)+".dat"), "w")
ref = open(os.path.join( out_path, "init_directions_"+str(k)+".dat"), "w")


# Load M mesh (centers of sphere mesh)
mv, mf = igl.read_triangle_mesh( os.path.join(data_path ,"centers.obj") ) 
center = vd.Mesh((mv, mf), alpha = 0.9, c=[0.4, 0.4, 0.81])


# Load test mesh
tv, tf = igl.read_triangle_mesh(os.path.join(data_path,  "test_remeshed_"+str(k)+".obj"))

# Create dual mesh
tmesh = Mesh()
tmesh.make_mesh(tv,tf)
# Get inner vertices
inner_vertices = tmesh.inner_vertices()
# Get outer vertices
outer_vertices = tmesh.boundary_vertices()

# Get vertex normals for test mesh
e_i = igl.per_vertex_normals(tv, tf)

# Fix normal directions
signs = np.sign(np.sum(e_i * ([0,0,1]), axis=1))
e_i = e_i * signs[:, None]

# Compute circumcenters and axis vectors for each triangle
p1, p2, p3 = tv[tf[:, 0]], tv[tf[:, 1]], tv[tf[:, 2]]

ct, rt, nt = circle_3pts(p1, p2, p3)

# Dual topology 
dual = tmesh.vertex_ring_faces_list()

# Dual topology without outer faces
dual_faces = [dual[vertex] for vertex in inner_vertices ]

# Create hexagonal mesh                            
h_pts = np.empty((len(tf), 3), dtype=np.float64)

# Intersect circumcircle axis with center mesh
for i in range(len(tf)):
    # Get points on circumcircle axis
    p0  = ct[i] - 10*nt[i]
    p1  = ct[i] + 10*nt[i]
    
    # Get intersection points
    h_pts[i,:] = np.array(center.intersect_with_line(p0, p1)[0])


# # Write data to file
for v  in tmesh.inner_vertices():
    file_pts.write(str(tmesh.vertices[v][0]) + " " + str(tmesh.vertices[v][1]) + " " + str(tmesh.vertices[v][2]) + "\n")
    ref.write(str(e_i[v][0]) + " " + str(e_i[v][1]) + " " + str(e_i[v][2]) + "\n")

file_pts.close()
ref.close()


# Optimization
# Init  X0 = [e_i | A| delta]
X = e_i.flatten() 

# Init constraints
linecong = LineCong()

linecong.initialize_constraint(X, len(tv), h_pts, dual, inner_vertices)
 
# Init optimizer
opt = Optimizer()
opt.initialize_optimizer(X)

for i in range(10):
    # Create file per iteration
    directions = open(os.path.join( out_path, "directions_"+str(k)+"_"+str(i)+".dat"), "w")

    # Compute J, r for each constraint
    linecong.compute(inner_vertices, dual, X)
    
    # Add constraints  
    opt.add_constraint(linecong)

    # Optimize
    opt.optimize("LM")

    e_i = opt.X.reshape(-1,3)   

    # Write data to file
    for v  in inner_vertices:   
        directions.write(str(e_i[v][0]) + " " + str(e_i[v][1]) + " " + str(e_i[v][2]) + "\n")
    directions.close()

# Print log
opt.print_log()
e_i = opt.X.reshape(-1,3)   
    
# Visualize line congruence
#congruence = vd.Lines(tv[inner_vertices[:5]]- 5*e_i[inner_vertices[:5]], tv[inner_vertices[:5]] + 5*e_i[inner_vertices[:5]], c=[0.8,0.5,0.5], lw=0.1)


# Torsal test

In [ ]:
def torsal_directions(v, f, e_i):
    num_faces = len(f)
    tors = np.zeros((2 * num_faces, 3))
    cos_tors = np.zeros(num_faces)
    for i in range(num_faces):
        # Extract vertex indices for the current face
        vertex_indices = f[i]

        # Calculate vectors e(u, v), e_u, e_v for the current face
        e_u, e_v = e_i[vertex_indices[1]] - e_i[vertex_indices[0]], \
                    e_i[vertex_indices[2]] - e_i[vertex_indices[0]]
        e = (e_i[vertex_indices[0]] + e_i[vertex_indices[1]] + e_i[vertex_indices[2]])/3

        # Calculate vectors a(u, v), a_u, a_v for the current face
        v_u, v_v = v[vertex_indices[1]] - v[vertex_indices[0]], \
                    v[vertex_indices[2]] - v[vertex_indices[0]]
        vv = (v[vertex_indices[0]] + v[vertex_indices[1]]+ v[vertex_indices[2]])/3

        # Calculate coefficients of the equation and solve it
        gamma_0 = np.linalg.det(np.array([e_u, v_u, e]))
        gamma_1 = np.linalg.det(np.array([e_u, v_v, e])) + np.linalg.det(np.array([e_v, v_u, e]))
        gamma_2 = np.linalg.det(np.array([e_v, v_v, e]))

        discr = gamma_1 * gamma_1 - 4.0 * gamma_0 * gamma_2
        # here should be exception about the negative discriminant

        # Negative discriminant
        if discr < 0 and abs(discr) > 1e-6:
            print("Discriminant is negative")
            tors[2*i] = np.array([0,0,0])
            tors[2 * i + 1] = np.array([0,0,0])
            cos_tors[i] = -1

        # Zero discriminant
        elif abs(discr) < 1e-6 :
            soln = -gamma_1
            sold = (2.0 * gamma_0)
                     
            # Calculate torsal directions for the current face
            tors1 = soln * v_u + sold * v_v
            tors1 = tors1 / np.linalg.norm(tors1)
            tors[2*i] = tors1
            tors[2 * i + 1] = tors1
            
            cos_tors[i] = 0

        # Positive discriminant
        else:            
            soln = [(-gamma_1 - np.sqrt(discr)), (-gamma_1 + np.sqrt(discr))]
            sold = (2.0 * gamma_0)
            
            # Calculate torsal directions for the current face
            tors1 = soln[0] * v_u + sold * v_v
            tors1 = tors1 / np.linalg.norm(tors1)
            tors[2*i] = tors1
            tors2 = soln[1] * v_u + sold * v_v
            tors2 = tors2 / np.linalg.norm(tors2)
            tors[2 * i + 1] = tors2

            normal_plane_1 = np.cross(tors1, e)
            normal_plane_2 = np.cross(tors2, e)
            
            cos_tors[i] = abs(np.dot(normal_plane_1, normal_plane_2)) / (np.linalg.norm(normal_plane_1) * np.linalg.norm(normal_plane_2))

        

    return tors, cos_tors

In [6]:
def torsal_dir_vec(tv, tf, e_i):
    
    # Get vertices
    vi, vj, vk = tv[tf[:,0]], tv[tf[:,1]], tv[tf[:,2]]  

    # Get directions
    ei, ej, ek = e_i[tf[:,0]], e_i[tf[:,1]], e_i[tf[:,2]]

    # Compute edges
    eij = ej - ei
    eik = ek - ei

    # Compute vectors
    vij = vj - vi
    vik = vk - vi

    # Compute barycenter of directions
    ec = np.sum( e_i[tf], axis = 1)/3

    # Compute barycenter of vertices
    barycenters = (vi + vj+ vk)/3

    # Cross products
    vijXec = np.cross(vij, ec)
    vikXec = np.cross(vik, ec)

    # Compute coefficients
    g0 = np.sum(eij*vijXec, axis=1)
    g1 = np.sum(eij*vikXec, axis=1) + np.sum(eik*vijXec, axis=1)
    g2 = np.sum(eik*vikXec, axis=1)

    # Compute discriminant
    disc = g1**2 - 4*g0*g2
    
    # Get indices with negative discriminant, positive discriminant and small discriminant (i.e almost zero)
    negative_disc = np.where((disc < 0) & (np.abs(disc) > 1e-6))
    small_disc = np.where((disc < 0) & (np.abs(disc) < 1e-6))
    pos_disc = np.where(disc >= 0)

    # Init torsal directions
    t1 = np.zeros((len(disc), 3))
    t2 = np.zeros((len(disc), 3))


    # For negative discriminant with absolute value > 1e-6
    t1[small_disc] = np.zeros((len(small_disc), 3))
    t2[small_disc] = np.zeros((len(small_disc), 3))

    # For negative discriminant with absolute value < 1e-6
    t1[negative_disc] = (-g1[negative_disc])[:, None] * vij[negative_disc] + (2 * g0[negative_disc])[:, None] * vik[negative_disc]
    t2[negative_disc] = t1[negative_disc]

    # For positive discriminant
    t1[pos_disc] = (-g1[pos_disc] + np.sqrt(disc[pos_disc]))[:, None] * vij[pos_disc] + (2 * g0[pos_disc])[:, None] * vik[pos_disc]
    t2[pos_disc] = (-g1[pos_disc] - np.sqrt(disc[pos_disc]))[:, None] * vij[pos_disc] + (2 * g0[pos_disc])[:, None] * vik[pos_disc]

    # Normalize
    t1[negative_disc] /= np.linalg.norm(t1[negative_disc], axis=1)[:, None]
    t2[negative_disc] /= np.linalg.norm(t2[negative_disc], axis=1)[:, None]

    t1[pos_disc] /= np.linalg.norm(t1[pos_disc], axis=1)[:, None]
    t2[pos_disc] /= np.linalg.norm(t2[pos_disc], axis=1)[:, None]

    # Init cosines vectors
    cos_tors = np.empty(len(disc))

    # Compute cross products
    t1Xec = np.cross(t1, ec)
    t2Xec = np.cross(t2, ec)

    # Get indices of nonzero vectors
    nonzeroInd = np.where(np.linalg.norm(t1Xec, axis=1) * np.linalg.norm(t2Xec, axis=1) >= 1e-6)

    # Compute cosines for nonzero vectors
    cos_tors[nonzeroInd] = np.sum( abs(t1Xec[nonzeroInd] * t2Xec[nonzeroInd]), axis=1) / (np.linalg.norm(t1Xec[nonzeroInd], axis=1) * np.linalg.norm(t2Xec[nonzeroInd], axis=1))

    # Compute cosines for zero vectors
    cos_tors[np.where(np.linalg.norm(t1Xec, axis=1) * np.linalg.norm(t2Xec, axis=1) < 1e-6)] = 0

    return  barycenters, t1, t2, cos_tors

In [58]:
# Check time
start = time.time()
bar, t1, t2, cos_tors = torsal_dir_vec(tv, tf, e_i)
end = time.time()
print(f"t1 : \n{t1[:5]} \n t2 :\n {t2[:5]} \n cos_tors :\n {cos_tors[:5]}")
print("Time for torsal directions Vec: ", end-start)

t1 : 
[[-0.96279169 -0.18559908 -0.19643101]
 [-0.97949548 -0.17996173 -0.09056697]
 [ 0.03652312  0.99468638  0.09625523]
 [ 0.88487614  0.465794   -0.00549166]
 [ 0.9998109   0.01834913  0.00643939]] 
 t2 :
 [[-0.3897405   0.91863064 -0.06496224]
 [-0.75444516  0.58154449  0.30433288]
 [-0.92537749  0.02619578 -0.37814056]
 [ 0.20355549  0.97675934  0.06712937]
 [ 0.58705273  0.80657832  0.06928571]] 
 cos_tors :
 [0.62571138 0.84745488 0.07734514 0.62433609 0.58821402]
Time for torsal directions Vec:  0.0045909881591796875


In [57]:
start = time.time()
tors, cos_tors = torsal_directions(tv, tf, e_i)
end = time.time()
# Get even indices
tors1 = tors[::2]
# Get odd indices
tors2 = tors[1::2]
print(f"t1 : \n{t1[:5]} \n t2 :\n {t2[:5]} \n cos_tors :\n {cos_tors[:5]}")
print("Time for torsal directions: ", end-start)

Discriminant is negative
Discriminant is negative
Discriminant is negative
Discriminant is negative
t1 : 
[[-0.96279169 -0.18559908 -0.19643101]
 [-0.97949548 -0.17996173 -0.09056697]
 [ 0.03652312  0.99468638  0.09625523]
 [ 0.88487614  0.465794   -0.00549166]
 [ 0.9998109   0.01834913  0.00643939]] 
 t2 :
 [[-0.3897405   0.91863064 -0.06496224]
 [-0.75444516  0.58154449  0.30433288]
 [-0.92537749  0.02619578 -0.37814056]
 [ 0.20355549  0.97675934  0.06712937]
 [ 0.58705273  0.80657832  0.06928571]] 
 cos_tors :
 [0.21686438 0.5883677  0.03060904 0.62433609 0.58821402]
Time for torsal directions:  0.3377056121826172


In [ ]:


tpos = open(os.path.join( out_path, "torsal_pos_"+str(k)+".dat"), "w")
tdir = open(os.path.join( out_path, "torsal_dir_nor_"+str(k)+".dat"), "w")

t_mesh = Mesh()
t_mesh.make_mesh(tv, tf)

inner_faces = t_mesh.inner_faces()

for ii in range(len(inner_faces)):
    i = inner_faces[ii]
    tpos.write(str(barycenters[i][0]) + " " + str(barycenters[i][1]) + " " + str(barycenters[i][2]) + "\n")
    tdir.write(str(torsal_dir[i][0]) + " " + str(torsal_dir[i][1]) + " " + str(torsal_dir[i][2]) + " " + str(torsal_dir[i][3]) + " " + str(torsal_dir[i][4]) + " " + str(torsal_dir[i][5]) + "\n")

tpos.close()
tdir.close()

igl.write_triangle_mesh(os.path.join( out_path, "tri_"+str(k)+".obj"), tv, tf)

In [ ]:
# Initialize polyscope
ps.init()


# Register a Mesh
v_tmesh = ps.register_surface_mesh("Mesh", tv, tf, smooth_shade=True)
ps.register_surface_mesh("Sphere Centers Mesh", h_pts, dual_faces, smooth_shade=True, edge_color=[0.8,0.5,0.5], edge_width=0.1)

# Draw circumcenter axis
# Register a point cloud
#v_cc = ps.register_point_cloud("Circumcenter ", tv, radius=0.01)

#print("ei :", e_i.shape)
#print("ct :", ct.shape)
# basic visualization
v_tmesh.add_vector_quantity("Axis", e_i, defined_on='vertices', enabled=True, radius=0.0001, length=5, color=(0.8, 0.0, 0.2))


# View the point cloud and mesh we just registered in the 3D UI
ps.show()

# Optimization Hyperbolicity

## Test 1 triangle

In [5]:
import igl
#import meshplot as mp
import vedo as vd
import polyscope as ps
import numpy as np
import os 
import time 
from geometry.mesh import Mesh
from geometry.utils import *
from optimization.Planarity import Planarity
from optimization.HyperbolicLC import HyperbolicLC
from optimization.Optimizer import Optimizer
from optimization.LineCong import LineCong

# tv test
tv = np.array([
    [0,0,1]  , # v0
    [1,0,0]  , # v1
    [0,1,0]  , # v2
    [0,0,-1] , # v3
    [-1,0,0] , # v4
    [0,-1,0] , # v5
    [-1,-1,0] # v6
    ])

# tf test
tf = np.array([
    [0,1,2],
    [0,2,3],
    [0,3,4],
    [0,4,5],
    [0,5,6]
    ])

# e_i test
e_i = np.array([
    [0,0,0.2],
    [0,0.4,0.4],
    [0.5,0.5,0.5],
    [0.5,0.5,-0.5],
    [0.5,-0.5,0.5],
    [0.5,-0.5,-0.5],
    [-0.5,0.0, 0.0]
    ])

e_i = e_i / np.linalg.norm(e_i, axis=1)[:, None]

# Compute the directions at the barycenters
ec = np.sum( e_i[tf], axis = 1) / 3

# Compute the norms of ec
#nc = np.linalg.norm(ec, axis=1)

# # Compute the edge vectors per each face
vi, vj, vk = tv[tf[:,0]], tv[tf[:,1]], tv[tf[:,2]]


# # Compute the edge vectors per each face
vij = vj - vi
vik = vk - vi

# Set up X 
#X[:3*self.nV] = V.flatten()

eij = e_i[tf[:,1]] - e_i[tf[:,0]]
eik = e_i[tf[:,2]] - e_i[tf[:,0]]

# A = [vij, eik, ec] + [eij, vik, ec], where [ , , ] denotes determinant
# A = det1 +  det2
eikXec = np.cross(eik, ec)
vikXec = np.cross(vik, ec)

det1 = np.sum(vij*eikXec, axis=1)
det2 = np.sum(eij*vikXec, axis=1)

# b = [eij, eik, ec]  c = [vij, vik, ec]

b = np.sum(eij*eikXec, axis=1)
c = np.sum(vij*vikXec, axis=1)

A = det1 + det2 

print(A**2 - 4*b*c - 1**2)

# variables = [e_i | A| delta]; e_i direction per vertex in T and A is one per each triangle in T, same for delta 
X = np.zeros(3*len(tv) + 2*len(tf) )

# Optimization
X[:3*len(tv)] = e_i.flatten()


# Init constraints ---------------------------------------------
# # Init LineCong
#linecong = LineCong()
# X variables, e_i dim, pts in hexagonal mesh, faces in hexagonal mesh, inner vertices indices.
#linecong.initialize_constraint(X, len(tv), h_pts, dual, inner_vertices)

# # Init Hyperbolic
hyp = HyperbolicLC()
w = 10
X = hyp.initialize_constraint(X, tv, tf, e_i, 5*np.random.rand(len(tf)), w)


# Init optimizer
opt = Optimizer()
opt.initialize_optimizer(X)

# Compute J, r for each constraint
#linecong.compute(inner_vertices, dual, X)

# Add constraints  
#opt.add_constraint(linecong)
time_opt = []

for i in range(20):

    # Take time
    start = time.time()
    hyp.compute(X, tf)

    opt.add_constraint(hyp)

    # Optimize
    opt.optimize("LM")
    end = time.time()

    #print(f"delta : {opt.X[-1]}")

    time_opt.append(end - start)


#opt.print_log()

#print(f"time:\n {time_opt}")

e_i = opt.X[:3*len(tv)].reshape(-1,3)

print(np.linalg.norm(e_i, axis=1))

eij = e_i[tf[:,1]] - e_i[tf[:,0]]
eik = e_i[tf[:,2]] - e_i[tf[:,0]]


ec = np.sum( e_i[tf], axis = 1) / 3
# A = [vij, eik, ec] + [eij, vik, ec], where [ , , ] denotes determinant
# A = det1 +  det2
eikXec = np.cross(eik, ec)
vikXec = np.cross(vik, ec)

det1 = np.sum(vij*eikXec, axis=1)
det2 = np.sum(eij*vikXec, axis=1)

# [vij, vik, ec] = det1 
vikXec = np.cross(vik, ec)
detA = np.sum(vij*vikXec, axis=1)

# [eij, eik, ec] = det2
eikXec = np.cross(eik, ec)
detC = np.sum(eij*eikXec, axis=1)
# b = [eij, eik, ec]  c = [vij, vik, ec]

A = det1 + det2 

Ac = opt.X[3*len(tv):3*len(tv)+len(tf)]

print(f"Disc f: {Ac**2 - 4*detA*detC} \t delta: { opt.X[3*len(tv) + len(tf):]} \tenergy: {(Ac**2 - 4*detA*detC - opt.X[3*len(tv) + len(tf):]**2)**2}")

print(f"Diff A {abs(Ac - A)}")

[ 1.42080693 -0.36068383 -0.77511864 -0.80246914 -0.94045574]
 E: 257.8167250925352
 E: 14.72983638347313
 E: 0.8912895148675313
 E: 0.02979588894166756
 E: 0.0011199663249491032
 E: 4.42385655839445e-05
 E: 1.7651897909526834e-06
 E: 7.057672930184627e-08
 E: 2.8229788546042963e-09
 E: 1.1292469677222028e-10
 E: 4.517283018763719e-12
 E: 1.8070372178468644e-13
 E: 7.228650078651619e-15
 E: 2.891660828553202e-16
 E: 1.1567455519500902e-17
 E: 4.627279892005178e-19
 E: 1.850971972741095e-20
 E: 7.40635178055316e-22
 E: 2.9613953848815483e-23
 E: 1.1838852849670456e-24
[1.01720016 1.51260252 1.33502083 1.10910236 1.20926208 1.25613585
 0.79436081]
Disc f: [8.26182494 0.32632998 0.06753881 1.44863483 0.19692355] 	 delta: [2.87433904 0.57125299 0.25988229 1.20359247 0.44376069] 	energy: [9.54521695e-27 1.63010710e-28 2.41588652e-30 3.70611291e-26
 9.47465858e-28]
Diff A [1.38777878e-15 8.88178420e-16 1.11022302e-15 4.27435864e-15
 5.99520433e-15]


## Test data

### Load Data

In [6]:
import igl
#import meshplot as mp
import vedo as vd
import polyscope as ps
import numpy as np
import os 
import time 
from geometry.mesh import Mesh
from geometry.utils import *
from optimization.Planarity import Planarity
from optimization.HyperbolicLC import HyperbolicLC
from optimization.Optimizer import Optimizer
from optimization.LineCong import LineCong

#vd.settings.default_backend = 'k3d'
# # Define paths
dir_path = os.getcwd()
data_path = dir_path+"/approximation/data/" # data path
out_path = dir_path+"/outputs/" # output path

# Data of interest
k = 2

# Init data files
file_pts = open(os.path.join( out_path, "points_"+str(k)+".dat"), "w")
ref = open(os.path.join( out_path, "init_directions_"+str(k)+".dat"), "w")

# Load M mesh (centers of sphere mesh)
mv, mf = igl.read_triangle_mesh( os.path.join(data_path ,"centers.obj") ) 

# Load test mesh
tv, tf = igl.read_triangle_mesh(os.path.join(data_path,  "test_remeshed_"+str(k)+".obj"))

# Create dual mesh
tmesh = Mesh()
tmesh.make_mesh(tv,tf)

# Get inner vertices
inner_vertices = tmesh.inner_vertices()

# Get vertex normals for test mesh
e_i = igl.per_vertex_normals(tv, tf)

# Fix normal directions
signs = np.sign(np.sum(e_i * ([0,0,1]), axis=1))
e_i = e_i * signs[:, None]

# Compute circumcenters and axis vectors for each triangle
p1, p2, p3 = tv[tf[:, 0]], tv[tf[:, 1]], tv[tf[:, 2]]

ct, rt, nt = circle_3pts(p1, p2, p3)

# Dual topology 
dual = tmesh.vertex_ring_faces_list()

# Dual topology without outer faces
dual_faces = [dual[vertex] for vertex in inner_vertices ]

# Create hexagonal mesh                            
h_pts = np.empty((len(tf), 3), dtype=np.float64)
center = vd.Mesh((mv, mf), alpha = 0.9, c=[0.4, 0.4, 0.81])

# Intersect circumcircle axis with center mesh
for i in range(len(tf)):
    # Get points on circumcircle axis
    p0  = ct[i] - 10*nt[i]
    p1  = ct[i] + 10*nt[i]
    
    # Get intersection points
    h_pts[i,:] = np.array(center.intersect_with_line(p0, p1)[0])

Mesh Data Structure: |V| = 469, |F| = 858, |E| = 1326


In [7]:
# Compute the norms of ec
#nc = np.linalg.norm(ec, axis=1)

# # Compute the edge vectors per each face
vi, vj, vk = tv[tf[:,0]], tv[tf[:,1]], tv[tf[:,2]]


# # Compute the edge vectors per each face
vij = vj - vi
vik = vk - vi

# Set up X 
#X[:3*self.nV] = V.flatten()

eij = e_i[tf[:,1]] - e_i[tf[:,0]]
eik = e_i[tf[:,2]] - e_i[tf[:,0]]

ec = np.sum( e_i[tf], axis = 1) / 3

# A = [vij, eik, ec] + [eij, vik, ec], where [ , , ] denotes determinant
# A = det1 +  det2
eikXec = np.cross(eik, ec)
vikXec = np.cross(vik, ec)

det1 = np.sum(vij*eikXec, axis=1)
det2 = np.sum(eij*vikXec, axis=1)

# b = [eij, eik, ec]  c = [vij, vik, ec]

b = np.sum(eij*eikXec, axis=1)
c = np.sum(vij*vikXec, axis=1)

A = det1 + det2 

#print((A**2 - 4*b*c)[:5])

# variables = [e_i | A| delta]; e_i direction per vertex in T and A is one per each triangle in T, same for delta 
X = np.zeros(3*len(tv) + 2*len(tf) )

# Optimization
X[:3*len(tv)] = e_i.flatten()


# Init constraints ---------------------------------------------
# # Init LineCong
#linecong = LineCong()
# X variables, e_i dim, pts in hexagonal mesh, faces in hexagonal mesh, inner vertices indices.
#linecong.initialize_constraint(X, len(tv), h_pts, dual, inner_vertices)

np.random.seed(0)
# # Init Hyperbolic
hyp = HyperbolicLC()
X = hyp.initialize_constraint(X, tv, tf, e_i, 1, 1)

# Init optimizer
opt = Optimizer()
opt.initialize_optimizer(X)

# Compute J, r for each constraint
#linecong.compute(inner_vertices, dual, X)

# Add constraints  
#opt.add_constraint(linecong)
time_opt = []

for i in range(20):

    # Take time
    start = time.time()
    hyp.compute(X, tf)

    opt.add_constraint(hyp)

    # Optimize
    opt.optimize("LM")
    end = time.time()

    #print(f"delta : {opt.X[-1]}")

    time_opt.append(end - start)


opt.print_log()

print(f"Av time:\n {np.mean(time_opt)}")

 E: 857.8286590197176
 E: 111.16449688669869
 E: 14.403132741395678
 E: 1.8652499377591805
 E: 0.2412258037329577
 E: 0.031079158357875707
 E: 0.0039646830179491084
 E: 0.0004942537446048583
 E: 5.902769386526986e-05
 E: 6.639628222005473e-06
 E: 7.010707055293444e-07
 E: 7.068210629547247e-08
 E: 7.194612497307359e-09
 E: 7.983983934422226e-10
 E: 1.459594294487115e-10
 E: 3.9882733933694543e-11
 E: 1.2115946567083534e-11
 E: 3.891960061176036e-12
 E: 1.1891600567513017e-12
 E: 4.148865785000761e-13
          Energy
0   8.578287e+02
1   1.111645e+02
2   1.440313e+01
3   1.865250e+00
4   2.412258e-01
5   3.107916e-02
6   3.964683e-03
7   4.942537e-04
8   5.902769e-05
9   6.639628e-06
10  7.010707e-07
11  7.068211e-08
12  7.194612e-09
13  7.983984e-10
14  1.459594e-10
15  3.988273e-11
16  1.211595e-11
17  3.891960e-12
18  1.189160e-12
19  4.148866e-13
Av time:
 0.2519834876060486


In [8]:
e_i = opt.X[:3*len(tv)].reshape(-1,3)

#print(np.linalg.norm(e_i, axis=1))

eij = e_i[tf[:,1]] - e_i[tf[:,0]]
eik = e_i[tf[:,2]] - e_i[tf[:,0]]


ec = np.sum( e_i[tf], axis = 1) / 3
# A = [vij, eik, ec] + [eij, vik, ec], where [ , , ] denotes determinant
# A = det1 +  det2
eikXec = np.cross(eik, ec)
vikXec = np.cross(vik, ec)
eijXec = np.cross(eij, ec)

det1 = np.sum(vij*eikXec, axis=1)
det2 = np.sum(eij*vikXec, axis=1)

# [vij, vik, ec] = det1 
vikXec = np.cross(vik, ec)
detA = np.sum(vij*vikXec, axis=1)

# [eij, eik, ec] = det2
eikXec = np.cross(eik, ec)
detC = np.sum(eij*eikXec, axis=1)
# b = [eij, eik, ec]  c = [vij, vik, ec]

A = det1 + det2 
A1 = det1 - np.sum(vik*eijXec, axis=1)

Ac = opt.X[3*len(tv):3*len(tv)+len(tf)]

neg = np.where(Ac**2 - 4*detA*detC  < 0)

# print meann time 
print(f"Av time:\n {np.mean(time_opt)}")
delta = opt.X[3*len(tv) + len(tf):]

print(f"Disc f: {(Ac**2 - 4*detA*detC)[neg[0][:5]]} \n num neg: {len(neg[0])} delta: { opt.X[-5:]} ")
print(f"energy hyp [:5]: {((Ac**2 - 4*detA*detC)[:5] - delta[:5]**2 - 0.2)**2} ")
print(f"First 5: disc {(Ac**2 - 4*detA*detC)[:5]}")
#print(f"energy A [:5]: {(Ac - det1 - det2)**2}")

print(f"Real A : {A[:5]} \t opt A : {Ac[:5]} ")
 

Av time:
 0.2519834876060486
Disc f: [-6.21704992e-08 -6.27273096e-08 -6.83529172e-08 -7.37049181e-08
 -2.93560525e-08] 
 num neg: 12 delta: [0.00171123 0.00321193 0.00048716 0.00460649 0.00445451] 
energy hyp [:5]: [0.04 0.04 0.04 0.04 0.04] 
First 5: disc [3.10266913e-04 3.62335905e-04 1.56525656e-04 5.40953075e-06
 5.04915005e-06]
Real A : [ 0.02176523  0.00132274  0.0240973   0.00122579 -0.00273068] 	 opt A : [ 0.02176523  0.00132274  0.0240973   0.00122579 -0.00273068] 


## Minimize Library test

In [19]:
from scipy import optimize


def energy(x0, ei_dim, cicj, dual_faces):

    # energy \sum_{f \in F} \sum_{cj,ci \in E(f)} || e_f (cj - ci)  ||^2
    cf = dual_faces
    
    # Init energy value
    total_energy = 0

    # Get directions
    ei = x0.reshape(ei_dim, 3)

    # Loop over faces
    for f in range(len(cf)):
        # Define Jacobian
        diff = np.dot(cicj[f], ei[f])

        # Define residual
        total_energy += np.sum( diff**2, axis=0)
    
    diff2 = np.sum(ei * ei, axis=1) - 1

    total_energy += np.sum(diff2**2, axis=0)

    return total_energy

# Define paths
dir_path = os.getcwd()
data_path = dir_path+"/approximation/data/" # data path
out_path = dir_path+"/outputs/" # output path

# Data of interest
k = 2

# Load M mesh (centers of sphere mesh)
mv, mf = igl.read_triangle_mesh( os.path.join(data_path ,"centers.obj") ) 
center = vd.Mesh((mv, mf), alpha = 0.9, c=[0.4, 0.4, 0.81])


# Load test mesh
tv, tf = igl.read_triangle_mesh(os.path.join(data_path,  "test_remeshed_"+str(k)+".obj"))

# Create test mesh
test_mesh =  vd.Mesh((tv, tf), alpha = 0.8)

# Draw edges Triangular mesh
edges = test_mesh.clone().lw(1.5).wireframe().flat().c('k').alpha(0.5)

# Create dual mesh
tmesh = Mesh()
tmesh.make_mesh(tv,tf)
# Get inner vertices
inner_vertices = tmesh.inner_vertices()
# Get outer vertices
outer_vertices = tmesh.boundary_vertices()

# Get vertex normals for test mesh
e_i = igl.per_vertex_normals(tv, tf)

# Fix normal directions
signs = np.sign(np.sum(e_i * ([0,0,1]), axis=1))
e_i = e_i * signs[:, None]

# Compute circumcenters and axis vectors for each triangle
p1, p2, p3 = tv[tf[:, 0]], tv[tf[:, 1]], tv[tf[:, 2]]

ct, rt, nt = circle_3pts(p1, p2, p3)

# Dual topology 
dual_faces = tmesh.dual_top()

# Dual topology without outer faces
#dual_faces = [dual[vertex] for vertex in inner_vertices ]

# Create hexagonal mesh                            
h_pts = np.empty((len(tf), 3), dtype=np.float64)

# Intersect circumcircle axis with center mesh
for i in range(len(tf)):
    # Get points on circumcircle axis
    p0  = ct[i] - 10*nt[i]
    p1  = ct[i] + 10*nt[i]
    
    # Get intersection points
    h_pts[i,:] = np.array(center.intersect_with_line(p0, p1)[0])

# Compute cij
cij = []
# Loop over faces
for f in range(len(dual_faces)):
    # Get face
    face = dual_faces[f]

    # Get vertices
    v0 = h_pts[face]
    v1 = np.roll(h_pts[face], -1, axis=0)
 
    cicj = (v1 - v0)/np.linalg.norm(v1 - v0, axis=1)[:, None]
    # Define direction
    cij.append(cicj)

dim = len(inner_vertices)


opt = optimize.minimize(energy, e_i[inner_vertices].flatten(), args=(dim, cij, dual_faces), options={'disp': True})


Mesh Data Structure: |V| = 469, |F| = 858, |E| = 1326
Optimization terminated successfully.
         Current function value: 0.379119
         Iterations: 33
         Function evaluations: 55178
         Gradient evaluations: 47


In [ ]:
e_i = opt.x.reshape(-1,3)

minfile = open(os.path.join( out_path, "min_directions_"+str(k)+".dat"), "w")

for v  in range(len(tmesh.inner_vertices())):   
    minfile.write(str(e_i[v][0]) + " " + str(e_i[v][1]) + " " + str(e_i[v][2]) + "\n")
minfile.close()



### Test Circumcircle

In [ ]:

# Get a list of random points
#p = np.random.rand(10,3)

p1 = np.random.rand(10,3)
p2 = np.random.rand(10,3)
p3 = np.random.rand(10,3)

c, rad, n = circle_3pts(p1,p2,p3)
print(f"centers: {c}\t rad: {rad}\t n: {n}")
circles = []
for ci in range(len(c)):
    print(f"centers: {c[ci]}\t rad: {rad[ci]}\t n: {n[ci]}")
    if ci%2 == 0:
        circles.append( vd.Circle(c[ci], r=rad[ci], c='red', alpha = 0.5).orientation(n[ci])) 
    else:
        circles.append( vd.Circle(c[ci], r=rad[ci], c="green", alpha = 0.5).orientation(n[ci])) 
pts1 = vd.Points(np.array( np.vstack((p1, p2, p3))), r = 10, c = 'black')
#pts2 = vd.Points(np.array([p1[1], p2[1], p3[1]]), r = 10, c = 'green')

vd.show(pts1,*circles)


## Sample Planarity Opt

In [ ]:
# Create four quads in 3D that merge in a vertex
v = np.array(
    [
        [ 0.01,  0.01,  0.8],
        [ 1.01,  0.03,  0.01],
        [-1.02,  0.02,  0.01],
        [ 0.01,  1.1, -0.2],
        [ 0.01, -1.3, -0.3],
        [-1.02,  1.01,  0.1],
        [ 1.01,  1.02,  0.2],
        [-1.04, -1.03, -0.3],
        [ 1.05, -1.04,  0.1],
    ]
   )
fcs = np.array([[0, 1, 6, 3], [2, 0, 3, 5], [7, 4, 0, 2], [4, 8, 1, 0]])

# Make mesh
m = Mesh()
m.make_mesh(v, fcs)

mesh1 = vd.Mesh((v, fcs), alpha = 0.8, c='r')

X = np.zeros((3*m.V+3*m.F), dtype=np.float64)

X[:3*m.V] = m.vertices.flatten()

# Init constraints
planarity = Planarity()
planarity.initialize_constraint(m, X)


# init optimizer
opt = Optimizer()

opt.initialize_optimizer(X)

for i in range(10):

    # Compute J, r for each constraint
    planarity.compute(m, X)
    
    # Add constraints  
    opt.add_constraint(planarity)
    opt.optimize("LM")
 

mesh2 = vd.Mesh((opt.X[:3*m.V].reshape(m.V,3),  fcs), alpha = 0.9, c='b')

print(opt.energy)

vd.show(mesh1, mesh2)



In [ ]:

v = np.array(
    [
        [ 0.01,  0.01,  0.8],
        [ 1.01,  0.03,  0.01],
        [-1.02,  0.02,  0.01],
        [ 0.01,  1.1, -0.2],
        [ 0.01, -1.3, -0.3],
        [-1.02,  1.01,  0.1],
        [ 1.01,  1.02,  0.2],
        [-1.04, -1.03, -0.3],
        [ 1.05, -1.04,  0.1],
    ]
   )
fcs = np.array([[0, 1, 6, 3], [2, 0, 3, 5], [7, 4, 0, 2], [4, 8, 1, 0]])

nv = np.array([[-0.08452795, -0.5184518 ,  0.1204968 ],
       [ 0.09325778, -0.5417252 , -0.14323919],
       [-0.39126414,  0.38199818,  0.21744937],
       [ 0.05301311,  0.4957555 , -0.02630325],
       [-0.04160952, -0.68063384,  0.11461049],
       [-0.64098644,  0.96492183,  0.3065665 ],
       [ 0.29607683,  0.33823755, -0.46029344],
       [-0.24634174, -0.29367885,  0.3622692 ],
       [ 0.5331171 , -1.4693466 , -0.43155265]]
       )


mesh1 = vd.Mesh((v, fcs), alpha = 0.8, c='r')
mesh2 = vd.Mesh((nv, fcs), alpha = 0.8, c='b')


vd.show(mesh1, mesh2)

In [ ]:
v, f = igl.read_triangle_mesh("models/catenoid_def_1.obj")
mesh = om.read_trimesh("models/catenoid_def_1.obj")
mp.plot(v, f, c=v[:, 0])

In [ ]:
def iso_stereo(n, h):
    # Isotropic model stereographic projection
    # n: normal vector
    # h: distance center to plane
    # return point in I3

    # Concatenate n[1:2] and h
    return np.vstack((n[1:2],v))/(1+ n[2])

In [ ]:
a = np.array([[1, 2, 3],[5,4,2] , [9,7,2]])
b = np.array([19,22,89])
# Add b as a column to a
c = np.column_stack((a,b))
print(a[:, 0:2])

In [ ]:
# Get face normals
nf = igl.per_face_normals(v, f, np.array([0.0, 0.0, 0.0]))
# Get indices v1 of each face
iv1 = f[:, 0]
# Get vertices of idv1
v1 = v[iv1, :]
# Compute dot product of each face normal with v1
h = np.sum(nf * v1, axis=1)

# Get Isotropic point per face
nv = np.column_stack((nf[:, 0:2], h)) / (1 + nf[:, 2, None])

nf = np.empty((0, 3), dtype=np.int32)

for fi in range(len(f)):
    aux = np.empty((0, ), dtype=np.int32)
    for adf in mesh.ff(mesh.face_handle(fi)):
        aux = np.hstack((aux, [adf.idx()]))
    if len(aux) == 3:
        nf = np.vstack((nf, aux))

mp.plot(nv, nf[:2])

In [ ]:
from scipy.sparse.linalg import spsolve
from scipy.sparse import csr_matrix

v, f = igl.read_triangle_mesh("models/Hall.obj")

# Draw the mesh
n = igl.per_vertex_normals(v, f) * 0.5 + 0.5
c = np.linalg.norm(n, axis=1)
p = mp.plot(v, f, c, return_plot=True) # plot

# Compute Cuvature properties
v1, v2, k1, k2 = igl.principal_curvature(v, f)



# Compute focal surfaces
rho1 = 100
rho2 = 1/k2

step = 0.001

lamb = 0.1 * np.ones(v.shape[0], dtype=np.float64)

J = np.eye(v.shape[0], dtype=np.float64) 

# Precalculate intermediate states
vs = [v]
cs = [c]
for i in range(30):
    # Compute the gradient of the focal surface
    H = csr_matrix(J.T@J)
    b = - J.T@(lamb - rho1)
    delta = spsolve(H, b)
    lamb = lamb + step * delta
    # Update the mesh
    v = v + lamb[:,None]*n

    # Recompute the normals
    n = igl.per_vertex_normals(v, f) * 0.5 + 0.5
    c = np.linalg.norm(n, axis=1)
    # Add to the list
    # p.update_object(vertices=v, colors=c)
    vs.append(v)
    cs.append(c)
    

# Add interactive visulization
@mp.interact(level=(0, 9))
def mcf(level=0):
    p.update_object(vertices=vs[level], colors=cs[level])
    vs.append(v)
    cs.append(c)
p